In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split, ConcatDataset, Subset

In [15]:
loaded_datasets_info = torch.load('/Users/jiaming/Desktop/Lab2/datas/saved_datasets_scaled.pth')
loaded_train_dataset = loaded_datasets_info['train_dataset']
loaded_val_dataset = loaded_datasets_info['val_dataset']
loaded_test_dataset = loaded_datasets_info['test_dataset']

In [16]:
batch_size = 128 
loaded_train_loader = DataLoader(loaded_train_dataset, batch_size = batch_size) 
loaded_val_loader = DataLoader(loaded_val_dataset, batch_size = batch_size) 
loaded_test_loader = DataLoader(loaded_test_dataset, batch_size = batch_size, shuffle=False) 

In [17]:
def metrics_output(preds,labels):
    true_labels = np.array(labels)
    predicted_probs = np.array(preds)
    binary_predictions = (predicted_probs >= 0.5).astype(int)
    auc = roc_auc_score(true_labels, predicted_probs)
    conf_matrix = confusion_matrix(true_labels, binary_predictions)
    tn, fp, fn, tp = conf_matrix.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = accuracy_score(true_labels, binary_predictions)
    f1 = f1_score(true_labels, binary_predictions)
    mcc = matthews_corrcoef(true_labels, binary_predictions)  
    return (auc, sensitivity, specificity, accuracy, f1, mcc)

In [27]:
learning_rate = 0.001
num_epochs = 100
# num_classes = 2

In [28]:
size = 8
class Net_conv(torch.nn.Module):
    def __init__(self, input_length):
        super(Net_conv, self).__init__()
        self.block_1 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=1,
                            out_channels=size,
                            kernel_size=1,
                            stride=1,
                            padding=0),
            torch.nn.BatchNorm1d(size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(in_channels=size,
                                out_channels=2*size,
                                kernel_size=3,
                                stride=1,
                                padding=1),
            torch.nn.BatchNorm1d(2*size)
        )
        self.block_2 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=2*size,
                            out_channels=4*size,
                            kernel_size=1,
                            stride=1,
                            padding=0),
            torch.nn.BatchNorm1d(4*size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(in_channels=4*size,
                                out_channels=2*size,
                                kernel_size=3, 
                                stride=1,
                                padding=1),
            torch.nn.BatchNorm1d(2*size)
        )
        iutput_size_block_1 = (input_length - 1 + 2 * 0) // 1 + 1  
        output_size_block_2 = (iutput_size_block_1 - 1 + 2 * 0) // 1 + 1  
        num_channels_last_layer = 2*size 
        linear_input_size = num_channels_last_layer * output_size_block_2    
        self.linear_1 = torch.nn.Linear(linear_input_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        shortcut = x.float()
        x = self.block_1(x)
        x = torch.nn.functional.relu(x + shortcut)    
        shortcut = x
        x = self.block_2(x)
        x = torch.nn.functional.relu(x + shortcut)     
        x = x.view(x.size(0), -1)
        x =  self.linear_1(x)
        x = self.sigmoid(x)
        return x

model = Net_conv(input_length = 24)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
criterion = nn.BCELoss()  

In [29]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_dataset = ConcatDataset([loaded_train_dataset, loaded_val_dataset])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

for fold_idx, (train_index, val_index) in enumerate(kf.split(train_loader.dataset)):
    predicted_probabilities = []
    true_labels = []
    train_loader_fold = DataLoader(Subset(train_loader.dataset, train_index), batch_size=batch_size)
    val_loader_fold = DataLoader(Subset(train_loader.dataset, val_index), batch_size=batch_size)
    model = Net_conv(input_length = 24)
    
    for epoch in range(num_epochs):
        model = model.train()
        for batch_idx, (features, targets) in enumerate(train_loader_fold):     
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, targets) 
            loss.backward()
            optimizer.step()
    
    with torch.set_grad_enabled(False):
        for batch_idx, (features, targets) in enumerate(val_loader_fold):
            logits = model(features.float())
            probabilities = logits
            predicted_probabilities.extend(probabilities.tolist())
            true_labels.extend(targets.tolist())
    #每一个都储存（没有必要，只要最后一次的就好）
    #torch.save(model.state_dict(), f'/Users/jiaming/Desktop/Lab2/datas/ROC/models/ResNet/ResNet_{fold_idx}.pth')
    #np.save(f'/Users/jiaming/Desktop/Lab2/datas/ROC/models/ResNet/resnet_y_val_pred_{fold_idx}.npy', predicted_probabilities)
    #np.save(f'/Users/jiaming/Desktop/Lab2/datas/ROC/models/ResNet/resnet_y_val_{fold_idx}.npy', true_labels)


In [30]:
# 只储存了最后一次
torch.save(model.state_dict(), '/Users/jiaming/Desktop/Lab2/datas/ROC/models/ResNet/ResNet.pth')

In [31]:
roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC = metrics_output(predicted_probabilities,true_labels)
print(roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC) 

0.5654761904761905 0.5612244897959183 0.5648148148148148 0.5631067961165048 0.5499999999999999 0.12589664547660617


In [32]:
# 只储存了最后一次
np.save('/Users/jiaming/Desktop/Lab2/datas/ROC/val/ResNet/resnet_y_val_pred.npy', predicted_probabilities)
np.save('/Users/jiaming/Desktop/Lab2/datas/ROC/val/ResNet/resnet_y_val.npy', true_labels)

In [33]:
predicted_probabilities = []  
true_labels = []  
with torch.set_grad_enabled(False): 
    for batch_idx, (features, targets) in enumerate(loaded_test_loader):
        logits = model(features.float())
        predicted_probabilities.extend(logits.tolist()) 
        true_labels.extend(targets.tolist())

In [34]:
roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC = metrics_output(predicted_probabilities,true_labels)
print(roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC) 

0.5592231842231843 0.45454545454545453 0.6190476190476191 0.5348837209302325 0.5 0.07456785876210716


In [35]:
np.save('/Users/jiaming/Desktop/Lab2/datas/ROC/test/ResNet/resnet_y_test_pred.npy', predicted_probabilities)
np.save('/Users/jiaming/Desktop/Lab2/datas/ROC/test/ResNet/resnet_y_test.npy', true_labels)